<a href="https://colab.research.google.com/github/pedrocivita/superComp_pedrotpc/blob/main/aula11/supercompAtividade11.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Atividade 11 - Supercomputação - Pedro Civita**

In [1]:
# No Colab, configure a GPU (T4 GPU, etc.) acessando "Runtime" -> "Change Runtime Type" -> GPU.
!nvidia-smi  # Verifique se a GPU está ativada

Wed Sep 18 17:47:25 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   58C    P8              10W /  70W |      0MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

## **Código - Aula 11**

In [19]:
%%writefile exercicio_variacao_diaria_memoria.cu

#include <thrust/device_vector.h>
#include <thrust/host_vector.h>
#include <thrust/transform.h>
#include <thrust/count.h>
#include <thrust/replace.h>
#include <thrust/reduce.h>
#include <iostream>
#include <fstream>
#include <chrono>
#include <cuda_runtime.h> // Inclui a biblioteca para consultas de memória da GPU

// Função customizada para calcular a diferença entre preços diários
struct diferenca_diaria {
    __host__ __device__
    double operator()(const double& preco_hoje, const double& preco_ontem) const {
        return preco_hoje - preco_ontem;
    }
};

// Função customizada para contar apenas valores positivos (aumento no preço)
struct aumento_positivo {
    __host__ __device__
    bool operator()(const double& x) const {
        return x > 0.0;
    }
};

// Função para consultar o status da memória da GPU
void checkGpuMemory() {
    size_t freeMem, totalMem;
    cudaMemGetInfo(&freeMem, &totalMem); // Obtém informações da memória da GPU

    std::cout << "Memória total da GPU: " << totalMem / (1024 * 1024) << " MB" << std::endl;
    std::cout << "Memória livre da GPU: " << freeMem / (1024 * 1024) << " MB" << std::endl;
    std::cout << "Memória usada pela GPU: " << (totalMem - freeMem) / (1024 * 1024) << " MB" << std::endl;
}

int main() {
    // Leitura do arquivo de dados
    std::ifstream file("stocks-google.txt");
    if (!file) {
        std::cerr << "Erro ao abrir o arquivo stocks-google.txt" << std::endl;
        return 1;
    }

    // Lê todos os valores do arquivo para um vetor host
    thrust::host_vector<double> host;
    double value;
    while (file >> value) {
        host.push_back(value);
    }

    int n = host.size();
    if (n < 2) {
        std::cerr << "O arquivo contém menos de dois valores!" << std::endl;
        return 1;
    }

    // Mostrar a memória inicial da GPU
    checkGpuMemory();

    // Transferir os dados para a GPU e medir o tempo de cópia
    auto start_copy = std::chrono::high_resolution_clock::now();
    thrust::device_vector<double> stocks = host;
    auto end_copy = std::chrono::high_resolution_clock::now();
    std::chrono::duration<double> duration_copy = end_copy - start_copy;
    std::cout << "Tempo de cópia para a GPU: " << duration_copy.count() << " segundos" << std::endl;

    // Mostrar a memória da GPU após a cópia dos dados
    checkGpuMemory();

    // Calcular a média total dos preços
    double soma_precos = thrust::reduce(stocks.begin(), stocks.end(), 0.0);
    double media_precos = soma_precos / n;
    std::cout << "Média dos preços das ações: " << media_precos << std::endl;

    // Vetor para armazenar a variação diária (um elemento a menos que stocks)
    thrust::device_vector<double> ganho_diario(n - 1);

    // Cálculo do ganho diário: ganho_diario[i] = stocks[i+1] - stocks[i]
    auto start_calc = std::chrono::high_resolution_clock::now();
    thrust::transform(stocks.begin() + 1, stocks.end(), stocks.begin(), ganho_diario.begin(), diferenca_diaria());
    auto end_calc = std::chrono::high_resolution_clock::now();
    std::chrono::duration<double> duration_calc = end_calc - start_calc;
    std::cout << "Tempo de cálculo da variação diária: " << duration_calc.count() << " segundos" << std::endl;

    // Contar os dias com aumento no preço (ganho positivo)
    int num_dias_com_aumento = thrust::count_if(ganho_diario.begin(), ganho_diario.end(), aumento_positivo());
    std::cout << "Dias com aumento no preço: " << num_dias_com_aumento << std::endl;

    // Substituir os valores negativos no ganho_diario por zero
    thrust::replace_if(ganho_diario.begin(), ganho_diario.end(), ganho_diario.begin(), thrust::placeholders::_1 < 0, 0.0);

    // Calcular o total dos aumentos (soma dos ganhos positivos)
    double soma_aumentos = thrust::reduce(ganho_diario.begin(), ganho_diario.end(), 0.0);

    // Calcular a média dos aumentos (somente dos dias em que o preço subiu)
    double media_aumentos = soma_aumentos / num_dias_com_aumento;
    std::cout << "Média dos aumentos nos dias com preço subindo: " << media_aumentos << std::endl;

    // Mostrar a memória final da GPU
    checkGpuMemory();

    return 0;
}

Writing exercicio_variacao_diaria_memoria.cu


In [20]:
!nvcc -arch=sm_75 -std=c++14 exercicio_variacao_diaria_memoria.cu -o exercicio_variacao_diaria_memoria

In [21]:
!./exercicio_variacao_diaria_memoria < stocks-google.txt

Memória total da GPU: 15102 MB
Memória livre da GPU: 14999 MB
Memória usada pela GPU: 103 MB
Tempo de cópia para a GPU: 0.000168332 segundos
Memória total da GPU: 15102 MB
Memória livre da GPU: 14997 MB
Memória usada pela GPU: 105 MB
Média dos preços das ações: 1580.08
Tempo de cálculo da variação diária: 3.713e-05 segundos
Dias com aumento no preço: 3056
Média dos aumentos nos dias com preço subindo: 3.15251
Memória total da GPU: 15102 MB
Memória livre da GPU: 14995 MB
Memória usada pela GPU: 107 MB


O uso da GPU mostrou-se altamente eficiente para cálculos paralelos, com memória utilizada de apenas 107 MB e tempo de cópia de 0.000168332 segundos. O cálculo da variação diária foi feito em 3.713e-05 segundos, processando grandes volumes de dados rapidamente, como a média dos preços (1580.08) e a contagem de dias com aumento de preço (3056). O uso de iteradores dinâmicos e funções de transformação otimiza o processamento, destacando a vantagem da GPU para operações complexas e de larga escala.